In [1]:
#!pip install pandas openai numpy matplotlib

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
pd.options.display.max_colwidth = None

In [4]:
fundus_descr = pd.read_csv(r"/home/jupyter/datasphere/project/fundus_descriptions_1020.csv")

In [5]:
valid_descr = fundus_descr[fundus_descr['is_valid']==True].copy()
valid_descr.head()

,is_valid,od_color,od_monotone,od_size,od_shape,od_border,od_excavation_size,od_excavation_location,od_excavation_ratio,od_vessels_location,od_pathology,vessels_art_course,vessels_art_turtuosity,vessels_art_bifurcation,vessels_art_caliber,vessels_vein_course,vessels_vein_turtuosity,vessels_vein_bifurcation,vessels_vein_caliber,vessels_ratio,vessels_pathology,macula_macular_reflex,macula_foveal_reflex,macula_pathology,peripheral_pathology,other_diagnosis,other_notes,text
0,True,бледно-розовый,True,нормальный,правильная,четкие,нормальный,в центре,NaN,в центре,.,нормальный,нормальная,нормальная,нормальный,нормальный,нормальная,нормальная,нормальный,NaN,NaN,нормальный,нормальный,слегка гиперпигментирована,участки разряжения пигмента,Альбинотическое глазное дно,NaN,"ДЗН бледно-розовый, границы четкие, ход и калибр сосудов не изменен. Макулярные рефлексы четки. Макула слегка гиперпигментирована. На периферии участки разряжения пигмента. Альбинотическое глазное дно."
1,True,бледно-розовый,True,нормальный,правильная,размытые,нормальный,в центре,NaN,в центре,NaN,нормальный,нормальная,нормальная,расширенный,нормальный,нормальная,нормальная,расширенный,NaN,NaN,нормальный,нормальный,светлый проминирующий очаг до 1 ДД кверху от МЗ,NaN,NaN,Детали не визуализируются.,"За флером: границы ДЗН нечеткие, сосуды расширены, кверху от МЗ светлый проминирующий очаг до 1 ДД, детали не визуализируются"
3,True,бледно-розовый,True,нормальный,правильная,четкие,нормальный,в центре,NaN,в центре,NaN,нормальный,нормальная,нормальная,нормальный,нормальный,нормальная,нормальная,нормальный,NaN,NaN,нормальный,нормальный,NaN,NaN,NaN,NaN,"ДЗН бледно-розовый, с четкими границами, ход и калибр сосудов не изменен, макулярный рефлекс ровный, четкий, видимая периферия без патологии."
4,True,бледно-розовый,True,нормальный,правильная,четкие,нормальный,в центре,.,наружный,.,нормальный,нормальная,нормальная,суженный,нормальный,извитая,нормальная,нормальный,.,.,гладкий,гладкий,.,аваскулярная зона и отслойка сетчатки в наружных и нижних отделах.,.,.,"ДЗН бледно-розовый, границы четкие, артерии несколько сужены, сосуды несколько извиты, смещены в височную сторону. Макуляные рефлексы сглажены, на периферии в наружних и нижних отделах (зона 2,3) аваскулярная зона, локальная отслойка сетчатки вдоль демаркационной линии"
5,True,бледный,False,меньше нормы,неправильная,размытые,нормальный,в центре,NaN,нижне-наружный,"Гипоплазия, тракционная деформация, фибро-васкулярный тяж, линия самоотграничения.",нормальный,нормальная,нормальная,нормальный,нормальный,нормальная,нормальная,нормальный,NaN,"Линия самоотграничения, аваскулярная сетчатка.",нормальный,нормальный,NaN,"Аваскулярная сетчатка, перераспределение пигмента.",NaN,NaN,"ДЗН бледный, гипопластичен, тракционно деформирован, от него в нижне-наружном направлении идет высокий фибро-васкулярный тяж, вдоль тяжа на сетчатке - линия самоотграничения. Остальная сетчатка аваскулярная. перераспределение пигмента"


In [6]:
valid_descr.iloc[137]['text']

'ДЗН бледно-розовый монотонный, границы четкие, ход сосудов не изменен, вены широкие, МЗ без патологии, в нижне-внутреннем квадранте преретинальный фиброз (организовавшийся гемофтальм) вокруг отграничен лазеркоагулятами, остальная периферия без особенностей'

In [7]:
valid_descr.shape

(891, 28)

Таким образом имеем 891 текстовое описание от реального врача, преобразованное в табличный формат. Реальное описание от врача хранится в колонке text. 

Преобразование в табличный формат осуществлялось за счёт Structured Outputs от OpenAI, и по идее все 891 описания должны быть валидными описаниями.

Но на втором же описании видим, что модель ошиблась, и из данного описания нельзя было выделять параметры, так как глазное дно не офтальмоскопируется:

In [8]:
valid_descr.iloc[1]

is_valid                                                                                                                                             True
od_color                                                                                                                                   бледно-розовый
od_monotone                                                                                                                                          True
od_size                                                                                                                                        нормальный
od_shape                                                                                                                                       правильная
od_border                                                                                                                                        размытые
od_excavation_size                                                          

Из-за таких косяков модели мы можем просесть в качестве обучающей выборки для LoRA адаптера.
Поэтому давайте проведём небольшой EDA по интересующим нас полям и оценим разнообразие существующих значений параметров.

# EDA

## OD

#### is_valid

In [9]:
is_valid_df = valid_descr['is_valid']
np.unique(is_valid_df)

array([ True])

Тут всё окей, действительно остались все те описания, которые модель посчитала валидными. Было бы странно увидеть другой результат.

#### od_color 

In [10]:
od_color_s = valid_descr['od_color']
np.unique(od_color_s)

array(['белый', 'бледно', 'бледно розовый', 'бледно-розовый',
       'бледно-розовый с сероватым оттенком',
       'бледно-розовый с серым оттенком',
       'бледно-розовый, с сероватым оттенком', 'бледноват', 'бледноватый',
       'бледный', 'бледный с серым оттенком',
       'бледный, с сероватым оттенком', 'близко-розовый',
       'деколорированный', 'розовый', 'серовато-бледно-розовый',
       'сероватый', 'серый'], dtype=object)

Видим некоторое количество значений цвета глазного дна, при этом в системе EYAS возможны только следующие цвета:
* Серый
* Белый
* Бледный
* Бледно-розовый
* Розовый
* Гиперемированный

In [11]:
od_color_s.value_counts()

бледно-розовый                          638
бледный                                 183
бледноват                                21
бледноватый                              13
серый                                    12
розовый                                  10
бледно розовый                            3
деколорированный                          1
серовато-бледно-розовый                   1
бледный, с сероватым оттенком             1
белый                                     1
близко-розовый                            1
бледно-розовый с серым оттенком           1
бледно-розовый с сероватым оттенком       1
сероватый                                 1
бледно-розовый, с сероватым оттенком      1
бледно                                    1
бледный с серым оттенком                  1
Name: od_color, dtype: int64

Пока переведём все цвета которые имеют сероватый оттенок в бледный или бледно-розовый. Деколорированный пока под вопросом. 

Таким образом, унифицируем наши значения в данных, чтобы они соответствовали системе EYAS. 

In [12]:
# Словарь соответствий
replacement_dict = {
    "бледноват": "бледный",
    "бледноватый": "бледный",
    "бледно розовый": "бледно-розовый",
    "бледно-розовый, с сероватым оттенком": "бледно-розовый", #бледный
    "сероватый": "серый",
    "бледно": "бледный",
    "бледно-розовый с серым оттенком": "бледно-розовый", #бледный
    "бледно-розовый с сероватым оттенком": "бледно-розовый", #бледный
    "близко-розовый": "бледно-розовый",
    "бледный, с сероватым оттенком": "бледный", #бледный
    "серовато-бледно-розовый": "серый", #бледный
    "бледный с серым оттенком": "серый" #серый
}

# Применение замены
od_color_s = od_color_s.replace(replacement_dict)

In [13]:
valid_descr.loc[:, 'od_color'] = od_color_s
valid_descr['od_color'].value_counts()

бледно-розовый      645
бледный             219
серый                15
розовый              10
деколорированный      1
белый                 1
Name: od_color, dtype: int64

#### od_monotone

Всё также вполне ожидаемо, менять значения нет нужды. Только посчитаем встречаемость.

In [14]:
od_monotone_s = valid_descr['od_monotone']
np.unique(od_monotone_s)

array([False,  True])

In [15]:
od_monotone_s.value_counts()

True     884
False      7
Name: od_monotone, dtype: int64

#### od_size

In [16]:
od_size_s = valid_descr['od_size']
np.unique(od_size_s)

array(['больше нормы', 'значительно уменьшен', 'меньше нормы',
       'нормальный', 'увеличен', 'увеличен в диаметре',
       'увеличенного размера'], dtype=object)

Набор значений отличается от доступных в системе:
* нормальный
* больше нормы
* меньше нормы

In [17]:
od_size_s.value_counts()

нормальный              847
меньше нормы             32
больше нормы              6
увеличен                  2
увеличен в диаметре       2
значительно уменьшен      1
увеличенного размера      1
Name: od_size, dtype: int64

Небольшое количество отличающихся значений, заменим их и добавим в оригинальный датафрейм.

In [18]:
# Словарь соответствий
replacement_dict = {
    "увеличен" : "больше нормы",
    "увеличен в диаметре" : "больше нормы",
    "увеличенного размера" : "больше нормы",
    "значительно уменьшен" : "меньше нормы",
}

# Применение замены
od_size_s = od_size_s.replace(replacement_dict)
valid_descr.loc[:, 'od_size'] = od_size_s
valid_descr['od_size'].value_counts()

нормальный      847
меньше нормы     33
больше нормы     11
Name: od_size, dtype: int64

#### od_shape

In [19]:
od_shape_s = valid_descr['od_shape']
np.unique(od_shape_s)

array(['аномальное строение', 'деформирован', 'неправильная',
       'нормальный', 'овальная', 'правильная', 'тракционно деформирован'],
      dtype=object)

Доступные в системе значения:
* правильная
* неправильная
* овальная

In [20]:
od_shape_s.value_counts()

правильная                 849
неправильная                25
овальная                    12
тракционно деформирован      2
деформирован                 1
аномальное строение          1
нормальный                   1
Name: od_shape, dtype: int64

In [21]:
# Словарь соответствий
replacement_dict = {
    "тракционно деформирован" : "неправильная",
    "деформирован" : "неправильная",
    "аномальное строение" : "неправильная",
    "нормальный" : "правильная",
}

# Применение замены
od_shape_s = od_shape_s.replace(replacement_dict)
valid_descr.loc[:, 'od_shape'] = od_shape_s
valid_descr['od_shape'].value_counts()

правильная      850
неправильная     29
овальная         12
Name: od_shape, dtype: int64

#### od_border

In [22]:
od_border_s = valid_descr['od_border']
np.unique(od_border_s)

array(['не прослеживаются', 'нечеткие', 'приподняты', 'размытые',
       'с внутренней стороны сглажены', 'сглажены', 'стушеванные',
       'тационно деформированный', 'четкие', 'чёткие'], dtype=object)

Значения в системе:
* четкие
* размытые

In [23]:
od_border_s.value_counts()

четкие                           753
размытые                         118
нечеткие                          11
с внутренней стороны сглажены      3
тационно деформированный           1
приподняты                         1
сглажены                           1
чёткие                             1
не прослеживаются                  1
стушеванные                        1
Name: od_border, dtype: int64

In [24]:
display(valid_descr[valid_descr["od_border"] == "стушеванные"]["text"])

755    ДЗН бледно-розовый, контуры стушеваны (прослеживаются), вены полнокровные, артерии сужены, макула – рефлексы сглажены, на периферии на 5 и 7 час. – преретинальные пленки ДЗН бледно-розовый, контуры стушеваны (прослеживаются), вены полнокровные, артерии сужены, макула – рефлексы сглажены, на периферии на 5 и 7 час. – преретинальные пленки
Name: text, dtype: object

In [25]:
# Словарь соответствий
replacement_dict = {
    "нечеткие" : "размытые",
    "с внутренней стороны сглажены" : "размытые",
    "тационно деформированный" : "размытые",
    "приподняты" : "размытые",
    "сглажены" : "размытые",
    "чёткие" : "четкие",
    "не прослеживаются" : "размытые",
    "стушеванные" : "размытые",
}

# Применение замены
od_border_s = od_border_s.replace(replacement_dict)
valid_descr.loc[:, 'od_border'] = od_border_s
valid_descr['od_border'].value_counts()

четкие      754
размытые    137
Name: od_border, dtype: int64

#### od_excavation_size

In [26]:
valid_descr['od_excavation_size'] = valid_descr['od_excavation_size'].astype(str)
od_excavation_size_s = valid_descr['od_excavation_size']
np.unique(od_excavation_size_s)

array(['0,9ДД', 'nan', 'Краевая', 'больше нормы', 'глаукоматозная',
       'глубокая', 'глубокая субтотальная', 'глубокий', 'краевая',
       'меньше нормы', 'нормальный', 'отсутствует', 'проминирует',
       'слегка проминирует', 'субтотальная'], dtype=object)

В системе доступные значения:
* Нормальный
* Больше нормы
* Меньше нормы

In [27]:
od_excavation_size_s.value_counts()

нормальный               862
больше нормы              11
глубокая                   4
глаукоматозная             2
субтотальная               2
проминирует                1
меньше нормы               1
nan                        1
0,9ДД                      1
глубокая субтотальная      1
отсутствует                1
слегка проминирует         1
краевая                    1
Краевая                    1
глубокий                   1
Name: od_excavation_size, dtype: int64

In [28]:
# Словарь соответствий
replacement_dict = {
    "глубокая" : "больше нормы",
    "глаукоматозная" : "больше нормы",
    "субтотальная" : "больше нормы",
    "проминирует" : "меньше нормы",
    "nan" : "нормальный",
    "0,9ДД" : "больше нормы",
    "глубокая субтотальная" : "больше нормы",
    "отсутствует" : "нормальный",
    "слегка проминирует" : "меньше нормы",
    "краевая" : "больше нормы",
    "Краевая" : "больше нормы",
    "глубокий" : "больше нормы",
}

# Применение замены
od_excavation_size_s = od_excavation_size_s.replace(replacement_dict)
valid_descr.loc[:, 'od_excavation_size'] = od_excavation_size_s
valid_descr['od_excavation_size'].value_counts()

нормальный      864
больше нормы     24
меньше нормы      3
Name: od_excavation_size, dtype: int64

#### od_excavation_location

In [29]:
valid_descr['od_excavation_location'] = valid_descr['od_excavation_location'].astype(str)
od_excavation_location_s = valid_descr['od_excavation_location']
np.unique(od_excavation_location_s)

array(['nan', 'в центре', 'височный', 'внутренний', 'краевая',
       'на периферии', 'наружный', 'нижне-внутренний', 'субтотальная',
       'тотальная'], dtype=object)

Системные значения:
* в центре
* верхний
* нижний
* наружный
* внутренний
* верхне-наружный
* верхне-внутренний
* нижне-наружный
* нижне-внутренний

In [30]:
od_excavation_location_s.value_counts()

в центре            881
наружный              2
височный              1
нижне-внутренний      1
nan                   1
внутренний            1
на периферии          1
тотальная             1
субтотальная          1
краевая               1
Name: od_excavation_location, dtype: int64

In [31]:
replacement_dict = {
    "височный": "наружный",
    "nan": "в центре",
    "на периферии": "наружный",
    "тотальная": "в центре",
    "субтотальная": "в центре",
    "краевая": "наружный"
}


# Применение замены
od_excavation_location_s = od_excavation_location_s.replace(replacement_dict)
valid_descr.loc[:, 'od_excavation_location'] = od_excavation_location_s
valid_descr['od_excavation_location'].value_counts()

в центре            884
наружный              5
нижне-внутренний      1
внутренний            1
Name: od_excavation_location, dtype: int64

#### od_excavation_ratio

In [32]:
valid_descr['od_excavation_ratio'] = valid_descr['od_excavation_ratio'].astype(str)
od_excavation_ratio_s = valid_descr['od_excavation_ratio']
np.unique(od_excavation_ratio_s)

array(['.', '.0', '.None', '.ov', '.дефолт', '.неизвестно', '/', '/None',
       '0,3', '0,3 ДД', '0,5', '0,6', '0,9', '0.3', '0.4-0.5', '0.5',
       '0.6', '0.7', '0.8', '0.9', ':', ': None', ':None', 'nan'],
      dtype=object)

In [33]:
od_excavation_ratio_s.value_counts()

nan            841
/                8
0.9              5
0.5              4
.None            4
0,5              3
0.3              3
0.8              3
0,9              2
:None            2
0.7              2
/None            2
0.6              1
.0               1
0,3 ДД           1
.дефолт          1
:                1
.ov              1
0,6              1
.неизвестно      1
.                1
0,3              1
0.4-0.5          1
: None           1
Name: od_excavation_ratio, dtype: int64

Возможные значения от 0 до 1

In [34]:
replacement_dict = {
    "/": "nan",
    ".None": "nan",
    "0,5": "0.5",
    "0,9": "0.9",
    ":None": "nan",
    "/None": "nan",
    ".0": "0",
    "0,3 ДД": "0.3",
    ".дефолт": "0.3",
    ":": "nan",
    ".ov": "nan", 
    "0,6": "0.6",
    ".неизвестно": "nan",
    ".": "nan",
    "0,3": "0.3",
    "0.4-0.5": "0.4",
    ": None": "nan"
}

# Применение замены
od_excavation_ratio_s = od_excavation_ratio_s.replace(replacement_dict)
valid_descr.loc[:, 'od_excavation_ratio'] = od_excavation_ratio_s

In [35]:
valid_descr['od_excavation_ratio'] = valid_descr['od_excavation_ratio'].astype(float)
valid_descr['od_excavation_ratio'].value_counts()

0.9    7
0.5    7
0.3    6
0.8    3
0.7    2
0.6    2
0.4    1
0.0    1
Name: od_excavation_ratio, dtype: int64

#### od_vessels_location

In [36]:
valid_descr['od_vessels_location'] = valid_descr['od_vessels_location'].astype(str)
od_vessels_location_s = valid_descr['od_vessels_location']
np.unique(od_vessels_location_s)

array(['nan', 'аномальное расположение', 'в центрe', 'в центре',
       'верхне-внутренний', 'верхний', 'височная', 'височный',
       'внутренний', 'на внутренней стороне', 'назально', 'наружный',
       'нижне-наружный', 'радиальный', 'радиальный ход сосудов',
       'смещение'], dtype=object)

Системные значения:
* в центре
* верхний
* нижний
* наружный
* внутренний
* верхне-наружный
* верхне-внутренний
* нижне-наружный
* нижне-внутренний

In [37]:
od_vessels_location_s.value_counts()

в центре                   829
наружный                    26
височный                    13
внутренний                   8
нижне-наружный               3
радиальный                   2
височная                     1
аномальное расположение      1
верхне-внутренний            1
в центрe                     1
верхний                      1
nan                          1
назально                     1
на внутренней стороне        1
радиальный ход сосудов       1
смещение                     1
Name: od_vessels_location, dtype: int64

In [38]:
replacement_dict = {
    "височный": "наружный",
    "радиальный": "в центре",
    "височная": "наружный",
    "nan": "в центре",
    "назально": "внутренний",
    "на внутренней стороне": "внутренний",
    "радиальный ход сосудов": "в центре",
    "в центрe": "в центре"
}

od_vessels_location_s = od_vessels_location_s.replace(replacement_dict)
valid_descr.loc[:, 'od_vessels_location'] = od_vessels_location_s
valid_descr['od_vessels_location'].value_counts()

в центре                   834
наружный                    40
внутренний                  10
нижне-наружный               3
аномальное расположение      1
верхне-внутренний            1
верхний                      1
смещение                     1
Name: od_vessels_location, dtype: int64

## Vessels

### Art

#### vessels_art_course

In [39]:
valid_descr['vessels_art_course'] = valid_descr['vessels_art_course'].astype(str)
vessels_art_course_s = valid_descr['vessels_art_course']
np.unique(vessels_art_course_s)

array(['аномальный', 'атипичный ход', 'в норме', 'височный',
       'височный сдвиг', 'внутреннее смещение', 'извитая',
       'наружное смещение', 'наружный', 'нормальное', 'нормальный',
       'радиальный', 'сдвиг височный', 'смещение внутрь',
       'смещение кнаружи', 'смещение наружу'], dtype=object)

Системные значения:
* нормальный
* смещение наружу
* смещение внутрь 

In [40]:
vessels_art_course_s.value_counts()

нормальный             849
смещение наружу          9
височный сдвиг           9
радиальный               4
нормальное               4
наружное смещение        3
смещение кнаружи         2
аномальный               2
извитая                  2
смещение внутрь          1
височный                 1
наружный                 1
внутреннее смещение      1
сдвиг височный           1
в норме                  1
атипичный ход            1
Name: vessels_art_course, dtype: int64

In [41]:
valid_descr[valid_descr['vessels_art_course'] == 'сдвиг височный']['text']

468    ДЗН бледный, монотонный, границы четкие, пигментированы. Сосуды сужены, височный сдвиг. От ДЗН идет складка сетчатки через МЗ к периферии, спаянная с плотной фиброзной мембраной в виде "паруса". Участки гиперпигментации вдоль складки. На периферии (зона 3) аваскулярный зоны
Name: text, dtype: object

Можно заметить, что модель иногда к ходу сосудов приписывает их извитость. Да и вообще в описания не разделяют на артерии и вены, просто пишут в общем о сосудах. 

In [42]:
valid_descr[valid_descr['vessels_art_course'] == 'извитая']['text']

741                                                                                                                                                                                            В верхне- наружном отделе складка сетчатки с пролиферативной тканьюидущая от ДЗН в верхне наружном направлении. ДЗН - эктопирован в темпоральную сторону, слегка прикрыт пролиферативной тканью. На периферии перераспредиление пигмента, более выраженные внутри ввиде аркады, периферичнее аваскулярные зоны. Сосуды извитые.
1019    ДЗН бледный, границы нечеткие, сосуды извиты, в МЗ твердый субретинальный эксудат, на периферии отложения субретинального эксудата, единичные ангиоматозно расширенные сосуды и микроаневризмы, пигментированные коагуляты, отграничивающие зоны эксудации ДЗН бледный, границы нечеткие, сосуды извиты, в МЗ твердый субретинальный эксудат, на периферии отложения субретинального эксудата, единичные ангиоматозно расширенные сосуды и микроаневризмы , пигментированные коагуляты, отграничи

При этом во 1 случае извитость сохраняется в параметре vessels_art_turtuosity и vessels_vein_turtuosity, а во 2 нет, подправим это там:

In [43]:
valid_descr[valid_descr['vessels_art_course'] == 'извитая'][['vessels_art_turtuosity', 'vessels_vein_turtuosity']]

,vessels_art_turtuosity,vessels_vein_turtuosity
741,извитая,извитая
1019,нормальная,изืน.}


In [44]:
valid_descr[valid_descr['vessels_art_course'] == 'извитая']['vessels_art_turtuosity']

741          извитая
1019     нормальная 
Name: vessels_art_turtuosity, dtype: object

In [45]:
valid_descr.loc[valid_descr['vessels_art_course'] == 'извитая', 'vessels_art_turtuosity'] = 'извитые'
valid_descr.loc[valid_descr['vessels_art_course'] == 'извитая', 'vessels_vein_turtuosity'] = 'извитые'

valid_descr[valid_descr['vessels_art_course'] == 'извитая'][['vessels_art_turtuosity', 'vessels_vein_turtuosity']]

,vessels_art_turtuosity,vessels_vein_turtuosity
741,извитые,извитые
1019,извитые,извитые


Вернёмся в ходу артерий

In [46]:
valid_descr[valid_descr['vessels_art_course'] == 'аномальный']['text']

206    ДЗН гипопластичен, аномальный выход сосудов, другие отделы видно плохо ( микрокоррия) ДЗН гипопластичен, аномальный выход сосудов, другие отделы видно плохо ( микрокоррия)
888                         ДЗН бледно-розовый, неправильной формы, в наружном сегменте серая ямка 0,3 ДД, границы четкие, аномальный ход сосудов, МЗ и периферия без особенностей
Name: text, dtype: object

In [47]:
replacement_dict = {
    "нормальное": "нормальный",
    "в норме": "нормальный",
    "наружное смещение": "смещение наружу",
    "смещение кнаружи": "смещение наружу",
    "височный сдвиг": "смещение наружу",
    "сдвиг височный": "смещение наружу",
    "височный": "смещение наружу",
    "наружный": "смещение наружу",
    "внутреннее смещение": "смещение внутрь",
    "радиальный": "нормальный",  # Радиальный ход чаще соответствует норме
    "извитая": "нормальный",
}

vessels_art_course_s = vessels_art_course_s.replace(replacement_dict)
valid_descr.loc[:, 'vessels_art_course'] = vessels_art_course_s
valid_descr['vessels_art_course'].value_counts()

нормальный         860
смещение наружу     26
смещение внутрь      2
аномальный           2
атипичный ход        1
Name: vessels_art_course, dtype: int64

#### vessels_art_turtuosity

In [48]:
valid_descr['vessels_art_turtuosity'] = valid_descr['vessels_art_turtuosity'].astype(str)
vessels_art_turtuosity_s = valid_descr['vessels_art_turtuosity']
np.unique(vessels_art_turtuosity_s)

array(['nan', 'в норме', 'извитая', 'извитые',
       'концентрически закручивающаяся складка от ДЗН к наружной периферии',
       'несколько извита', 'нормальная'], dtype=object)

Системные значения:
* нормальная
* извитые
* прямые

In [49]:
vessels_art_turtuosity_s.value_counts()

нормальная                                                            800
извитая                                                                83
несколько извита                                                        3
извитые                                                                 2
концентрически закручивающаяся складка от ДЗН к наружной периферии      1
nan                                                                     1
в норме                                                                 1
Name: vessels_art_turtuosity, dtype: int64

In [50]:
replacement_dict = {
    "извитая": "извитые",
    "несколько извита": "извитые",
    "концентрически закручивающаяся складка от ДЗН к наружной периферии": "извитые",
    "nan": "нормальная",
    "в норме": "нормальная",
}

vessels_art_turtuosity_s = vessels_art_turtuosity_s.replace(replacement_dict)
valid_descr.loc[:, 'vessels_art_turtuosity'] = vessels_art_turtuosity_s
valid_descr['vessels_art_turtuosity'].value_counts()

нормальная    802
извитые        89
Name: vessels_art_turtuosity, dtype: int64

#### vessels_art_bifurcation

In [51]:
valid_descr['vessels_art_bifurcation'] = valid_descr['vessels_art_bifurcation'].astype(str)
vessels_art_bifurcation_s = valid_descr['vessels_art_bifurcation']
np.unique(vessels_art_bifurcation_s)

array([' ', 'nan', 'в норме', 'нормальная', 'под острым углом'],
      dtype=object)

Системные значения: 
* нормальная
* под острым углом
* под тупым углом

In [52]:
valid_descr[valid_descr['vessels_art_bifurcation']=='в норме']['text']

592    ДЗН аномальный, увеличенного размера, бледный, с сероватым оттенком, с четкими границами. Краевая ЭД. Сосуды смещены назально, резко сужены. MZ – не дифференцируется. Сетчатка прилежит. ДЗН аномальный, увеличенного размера, бледный, с сероватым оттенком, с четкими границами. Краевая ЭД. Сосуды смещены назально, резко сужены. MZ – не дифференцируется. Сетчатка прилежит.
Name: text, dtype: object

Можно заметить, что во всех 4 случая с остым углом бифуркации модель не ошиблась, то есть это не ошибки False Positive, но может где-то есть False Negative

In [53]:
valid_descr[valid_descr['vessels_art_bifurcation']=='под острым углом']['text']

373                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      ДЗН бледный, тракционно деформирован, угол выхода сосудов сужен, макула деформирована, смещена в височную сторону, рефлексы сглажены, на периферии множественные пигментированные сливные коагуляты
374                                                                                                                                                                                                                                                                                                        

In [54]:
vessels_art_bifurcation_s.value_counts()

нормальная          884
под острым углом      4
nan                   1
в норме               1
                      1
Name: vessels_art_bifurcation, dtype: int64

In [55]:
replacement_dict = {
    "nan": "нормальная",
    "в норме": "нормальная",
    " ": "нормальная"
}

vessels_art_bifurcation_s = vessels_art_bifurcation_s.replace(replacement_dict)
valid_descr.loc[:, 'vessels_art_bifurcation'] = vessels_art_bifurcation_s
valid_descr['vessels_art_bifurcation'].value_counts()

нормальная          887
под острым углом      4
Name: vessels_art_bifurcation, dtype: int64

#### vessels_art_caliber

In [56]:
valid_descr['vessels_art_caliber'] = valid_descr['vessels_art_caliber'].astype(str)
vessels_art_caliber_s = valid_descr['vessels_art_caliber']
np.unique(vessels_art_caliber_s)

array([" нормальный'vessels_vein_course':' Отграниченной к протоку'  annouvé_adjunctevessels_art_ratio_minor,' ",
       'ангиоматозно расширенный', 'ангиоматозноразширенный', 'нарушен',
       'несколько сужены', 'нормальный', 'полнокровны', 'полнокровные',
       'полнокровный', 'расширенные', 'расширенный', 'резко расширенный',
       'резко суженные', 'резко суженный', 'резко сужены', 'суженный',
       'умеренно суженный', 'утолщенный'], dtype=object)

Системные значения: 
* нормальный 
* расширенный 
* суженный

In [57]:
vessels_art_caliber_s.value_counts()

нормальный                                                                                                 695
суженный                                                                                                   142
расширенный                                                                                                 33
полнокровны                                                                                                  3
несколько сужены                                                                                             2
резко суженный                                                                                               2
резко сужены                                                                                                 2
полнокровный                                                                                                 2
расширенные                                                                                                  1
р

In [58]:
valid_descr[valid_descr['vessels_art_caliber']==""" нормальный'vessels_vein_course':' Отграниченной к протоку'  annouvé_adjunctevessels_art_ratio_minor,' """]['text']

1019    ДЗН бледный, границы нечеткие, сосуды извиты, в МЗ твердый субретинальный эксудат, на периферии отложения субретинального эксудата, единичные ангиоматозно расширенные сосуды и микроаневризмы, пигментированные коагуляты, отграничивающие зоны эксудации ДЗН бледный, границы нечеткие, сосуды извиты, в МЗ твердый субретинальный эксудат, на периферии отложения субретинального эксудата, единичные ангиоматозно расширенные сосуды и микроаневризмы , пигментированные коагуляты, отграничивающие зоны эксудации
Name: text, dtype: object

In [59]:
valid_descr[valid_descr['vessels_art_caliber']=='утолщенный']['text']

1010    ДЗН проминирует, бледно розовый с четкими границами. Сосуды извиты и утолщены Макула и сетчатка без очаговой патологии. ДЗН проминирует, бледно розовый с четкими границами. Сосуды извиты и утолщены Макула и сетчатка без очаговой патологии.
Name: text, dtype: object

In [60]:
valid_descr[valid_descr['vessels_art_caliber']=='нарушен']['text']

839    Над областью ДЗН, покрывая его, визуализируется проминирующий очаг - фиброзная ткань, крепящаяся парапапиллярно. Границы ДЗН под тканью не прослеживаются ход калибр сосудов нарушен- они извиты, макула по возрасту, очаговых симптомов нет Над областью ДЗН, покрывая его, визуализируется проминирующий очаг - фиброзная ткань, крепящаяся парапапиллярно. Границы ДЗН под тканью не прослеживаются ход калибр сосудов нарушен- они извиты, макула по возрасту, очаговых симптомов нет
Name: text, dtype: object

In [61]:
replacement_dict = {
    "полнокровны": "расширенный",
    "полнокровный": "расширенный",
    "полнокровные": "расширенный",
    "расширенные": "расширенный",
    "резко расширенный": "расширенный",
    "ангиоматозно расширенный": "расширенный",
    "ангиоматозноразширенный": "расширенный",
    "несколько сужены": "суженный",
    "резко суженный": "суженный",
    "резко сужены": "суженный",
    "резко суженные": "суженный",
    "умеренно суженный": "суженный",
    "утолщенный": "расширенный",
    """ нормальный'vessels_vein_course':' Отграниченной к протоку'  annouvé_adjunctevessels_art_ratio_minor,' """: "расширенный"
}

vessels_art_caliber_s = vessels_art_caliber_s.replace(replacement_dict)
valid_descr.loc[:, 'vessels_art_caliber'] = vessels_art_caliber_s
valid_descr['vessels_art_caliber'].value_counts()

нормальный     695
суженный       150
расширенный     45
нарушен          1
Name: vessels_art_caliber, dtype: int64

### Vein

#### vessels_vein_course

In [62]:
valid_descr['vessels_vein_course'] = valid_descr['vessels_vein_course'].astype(str)
vessels_vein_course_s = valid_descr['vessels_vein_course']
np.unique(vessels_vein_course_s)

array(['  insumواک парамет师יית="#ff---기.}.', 'аномальный',
       'атипичный ход', 'в норме', 'височный', 'височный сдвиг',
       'внутреннее смещение', 'извитая', 'наружное смещение',
       'нормальное', 'нормальный', 'радиальный', 'смещение кнаружи',
       'смещение наружу'], dtype=object)

Системные значения:
* нормальный
* смещение наружу
* смещение внутрь 

In [63]:
vessels_vein_course_s.value_counts()

нормальный                            865
височный сдвиг                          7
нормальное                              4
радиальный                              3
внутреннее смещение                     2
смещение наружу                         2
наружное смещение                       1
височный                                1
смещение кнаружи                        1
аномальный                              1
в норме                                 1
извитая                                 1
атипичный ход                           1
  insumواک парамет师יית="#ff---기.}.      1
Name: vessels_vein_course, dtype: int64

In [64]:
valid_descr[valid_descr['vessels_vein_course'] == 'аномальный']['text']

206    ДЗН гипопластичен, аномальный выход сосудов, другие отделы видно плохо ( микрокоррия) ДЗН гипопластичен, аномальный выход сосудов, другие отделы видно плохо ( микрокоррия)
Name: text, dtype: object

In [65]:
valid_descr[valid_descr['vessels_vein_course'] == '  insumواک парамет师יית=\"#ff---기.}.']['text']

1019    ДЗН бледный, границы нечеткие, сосуды извиты, в МЗ твердый субретинальный эксудат, на периферии отложения субретинального эксудата, единичные ангиоматозно расширенные сосуды и микроаневризмы, пигментированные коагуляты, отграничивающие зоны эксудации ДЗН бледный, границы нечеткие, сосуды извиты, в МЗ твердый субретинальный эксудат, на периферии отложения субретинального эксудата, единичные ангиоматозно расширенные сосуды и микроаневризмы , пигментированные коагуляты, отграничивающие зоны эксудации
Name: text, dtype: object

In [66]:
replacement_dict = {
    "нормальный": "нормальный",
    "нормальное": "нормальный",
    "в норме": "нормальный",
    "радиальный": "нормальный",  # Радиальный ход часто является вариантом нормы
    "извитая": "нормальный",
    "височный сдвиг": "смещение наружу",
    "височный": "смещение наружу",
    "наружное смещение": "смещение наружу",
    "смещение кнаружи": "смещение наружу",
    "внутреннее смещение": "смещение внутрь",
    "смещение внутрь": "смещение внутрь",
    "  insumواک парамет师יית=\"#ff---기.}.": "нормальный"
}


vessels_vein_course_s = vessels_vein_course_s.replace(replacement_dict)
valid_descr.loc[:, 'vessels_vein_course'] = vessels_vein_course_s
valid_descr['vessels_vein_course'].value_counts()

нормальный         875
смещение наружу     12
смещение внутрь      2
аномальный           1
атипичный ход        1
Name: vessels_vein_course, dtype: int64

In [67]:
valid_descr[valid_descr['vessels_vein_course'] == 'смещение внутрь']['text']

64     ДЗН тракционно деформирован, бледноват, сосуды смещены в носовую сторону, экстраретинальная ткань в центральных отделах, макула смещена, шизис в МЗ, на периферии остаточные аваскулярные зоны
360           ДЗН тракционно деформирован, височный сдвиг сосудов, в центральной зоне хориоретинальные очаги, преретинальный фиброз, сетчатка бледновата, плоская складка сетчатки в центральной зоне
Name: text, dtype: object

В случае описания 360 модель ошиблась и там бы следовало указать смещение наружу, так как есть височный сдвиг, исправим это отдельно.

In [68]:
valid_descr.loc[360, 'vessels_vein_course'] = 'смещение наружу'
valid_descr.loc[360, 'vessels_art_course'] = 'смещение наружу'

Наблюдаем разницу в распределении значений хода вен и артерий, поэтому посмотрим явно глазами на эти показатели:

In [69]:
valid_descr[valid_descr['vessels_vein_course'] != valid_descr['vessels_art_course']][['vessels_art_course', 'vessels_vein_course',  'text']]

,vessels_art_course,vessels_vein_course,text
22,смещение наружу,нормальный,"ДЗН бледно, смещение сосудов в височную сторону, макула эктопична, гипопластична, на периферии- коагуляты, небольшое количество преретинальных пленок. ДЗН бледно, смещение сосудов в височную сторону, макула эктопична, гипопластична, на периферии- коагуляты, небольшое количество преретинальных пленок."
51,смещение наружу,нормальный,"ДЗН бледный, монотонный, границы не четкие, височный сдвиг сосудов, от ДЗН кнаружи кверху складка сетчатки, самоотграниченная"
63,нормальный,смещение наружу,"ДЗН бледный, монотонный, границы четкие, пигментированы. Сосуды сужены, височный сдвиг. От ДЗН идет складка сетчатки через МЗ к периферии, спаянная с плотной фиброзной мембраной в виде ""паруса"". Участки гиперпигментации вдоль складки. На периферии аваскулярный зоны"
86,смещение наружу,нормальный,"ДЗН бледно-розовый, монотонный, тракционно деформирован, границы четкие, височные сдвиг сосудов, МЗ сглажена, эктопирована кнаружи, на периферии остаточная авасткулярная зона с пигментированными коагулятами, на границе с которой зона тракционного шизиса ДЗН бледно-розовый, монотонный, тракционно деформирован, границы четкие, височные сдвиг сосудов, МЗ сглажена, эктопирована кнаружи, на периферии остаточная авасткулярная зона с пигментированными коагулятами, на границе с которой зона тракционного шизи са"
100,смещение наружу,нормальный,"ДЗН бледный, монотонный, границы четкие, перипапиллярная пигментация, височный сдвиг сосудов, складка сетчатки, идущая от ДЗН через МЗ, расширяющаяся к наружной периферии с высокой экстраретинальной тканью"
188,смещение наружу,нормальный,"ДЗН бледно-розовый, монотонный, границы четкие, височный сдвиг сосудов, от ДЗН на наружную периферию идет складка сетчатки, МЗ не дифференцируется, на периферии остаточные аваскулярные зоны, преретинальный фиброз ДЗН бледно-розовый, монотонный, границы четкие, височный сдвиг сосудов, от ДЗН на наружную периферию идет складка сетчатки, МЗ не дифференцируется, на периферии остаточные аваскулярные зоны, преретинальный фиброз"
233,смещение наружу,нормальный,"ДЗН бледный, монотонный, границы четкие, парапапиллярная пигментация, височный сдвиг сосудов, макулярные рефлексы не визуализируются, складка сетчатки, идущая от ДЗН к 7ч, отложения пигмента вдоль складки, на периферии остаточные аваскулярные зоны"
302,смещение наружу,нормальный,"ДЗН бледный, монотонный, границы четкие, височный сдвиг сосудов, от ДЗН кверху-кнаружи идет высокая складка сетчатки с сосудами и преретинальным фиброзом, отложение пигмента в виде линии самоотграничения перед складкой, центральные рефлексы не определяются, остальная сетчатка обеднена сосудами, на периферии остаточные аваскулярные зоны, множественные пигментированные лазеркоагуляты ДЗН бледный, монотонный, границы четкие, височный сдвиг сосудов, от ДЗН кверху-кнаружи идет высокая складка сетчатки с сосудами и преретинальным фиброзом, отложение пигмента в виде линии самоотграничения перед складкой, центральные рефлексы не определяются, остальная сетчатка обеднена сосудами, на периферии остаточные аваскулярные зоны, множественные пигментированные лазеркоагуляты"
352,смещение наружу,нормальный,"ДЗН бледно-розовый, монотонный, границы четкие, парапапиллярная пигментация, височный сдвиг сосудов (угол выхода сосудов 20гр), МЗ не дифференцируется, от ДЗН к наружней периферии идет нежный тяж, видимая периферия без особенностей"
468,смещение наружу,нормальный,"ДЗН бледный, монотонный, границы четкие, пигментированы. Сосуды сужены, височный сдвиг. От ДЗН идет складка сетчатки через МЗ к периферии, спаянная с плотной фиброзной мембраной в виде ""паруса"". Участки гиперпигментации вдоль складки. На периферии (зона 3) аваскулярный зоны"


Практически во всех случая правильно определено смещение артерий, а не вен (должно быть смещение наружу). Во всех этих случаях в текстовых описаниях написано в общем о сосудах. 

При этом есть одна ошибка, где у обоих параметров должно быть значение нормально -- 629.

Под вопросом остаётся только случаях 888, так как там сказано про аномальный ход сосудов, не очень ясно, что это такое.

Аномальный случай пока оставим, а в остальных заменим.

In [70]:
valid_descr.loc[[22, 51, 63, 86, 100, 188, 233, 302, 352, 468, 566, 630, 724, 798, 896, 898, 993], 'vessels_vein_course'] = 'смещение наружу'
valid_descr.loc[[22, 51, 63, 86, 100, 188, 233, 302, 352, 468, 566, 630, 724, 798, 896, 898, 993], 'vessels_art_course'] = 'смещение наружу'

In [71]:
valid_descr.loc[629, 'vessels_vein_course'] = 'нормальный'
valid_descr.loc[629, 'vessels_art_course'] = 'нормальный'

In [72]:
valid_descr[valid_descr['vessels_vein_course'] != valid_descr['vessels_art_course']][['vessels_art_course', 'vessels_vein_course',  'text']]

,vessels_art_course,vessels_vein_course,text
888,аномальный,нормальный,"ДЗН бледно-розовый, неправильной формы, в наружном сегменте серая ямка 0,3 ДД, границы четкие, аномальный ход сосудов, МЗ и периферия без особенностей"


#### vessels_vein_turtuosity

In [73]:
valid_descr['vessels_vein_turtuosity'] = valid_descr['vessels_vein_turtuosity'].astype(str)
vessels_vein_turtuosity_s = valid_descr['vessels_vein_turtuosity']
np.unique(vessels_vein_turtuosity_s)

array(['nan', 'Извитая', 'в норме', 'извитая', 'извитые',
       'несколько извита', 'нормальная', 'полнокровны',
       'умеренно извитая'], dtype=object)

Системные значения:
* нормальная
* извитые
* прямые

In [74]:
valid_descr[valid_descr['vessels_vein_turtuosity'] == 'полнокровны']['text']

192    ДЗН бледно-розовый с четкими границами. Вены центральной области полнокровны, артерии нормального калибра . Макулярный рефлекс сглажен. На крайней периферии снаружи в области 2-3 час. ПВХРД в виде "следа улитки", не полностью отгранич коагул. Преретинальный фиброз в области бывших аваскулярных зон . Аномальный ход сосудов на крайней периферии.
Name: text, dtype: object

In [75]:
valid_descr[valid_descr['vessels_vein_turtuosity'] == 'Извитая']['text']

1012    Д.з.н. – бледно-розовый, контуры четкие, в макулярной области субретинальный желтоватый экссудат, в виде «фигуры звезды», субретинальный экссудат в парамакулярной области и на средней периферии, в наружном квадранте извитые сосуды с множественными аневризмами Д.з.н. – бледно-розовый, контуры четкие, в макулярной области субретинальный желтоватый экссудат, в виде «фигуры звезды», субретинальный экссудат в парамакулярной области и на средней периферии, в наружном квадранте извитые сосуды с множественными аневризмами
Name: text, dtype: object

In [76]:
vessels_vein_turtuosity_s.value_counts()

нормальная          801
извитая              80
несколько извита      3
извитые               2
полнокровны           1
nan                   1
в норме               1
умеренно извитая      1
Извитая               1
Name: vessels_vein_turtuosity, dtype: int64

In [77]:
replacement_dict = {
    "извитая": "извитые",
    "несколько извита": "извитые",
    "полнокровны": "нормальная",
    "nan": "нормальная",
    "в норме": "нормальная",
    "умеренно извитая": "извитые",
    "Извитая": "извитые"
}

vessels_vein_turtuosity_s = vessels_vein_turtuosity_s.replace(replacement_dict)
valid_descr.loc[:, 'vessels_vein_turtuosity'] = vessels_vein_turtuosity_s
valid_descr['vessels_vein_turtuosity'].value_counts()

нормальная    804
извитые        87
Name: vessels_vein_turtuosity, dtype: int64

В некоторых описаниях извитость вен и артерий не совпадает, посмотрим на эти записи:

In [78]:
valid_descr[valid_descr['vessels_vein_turtuosity'] != valid_descr['vessels_art_turtuosity']][['vessels_art_turtuosity', 'vessels_vein_turtuosity',  'text']]

,vessels_art_turtuosity,vessels_vein_turtuosity,text
4,нормальная,извитые,"ДЗН бледно-розовый, границы четкие, артерии несколько сужены, сосуды несколько извиты, смещены в височную сторону. Макуляные рефлексы сглажены, на периферии в наружних и нижних отделах (зона 2,3) аваскулярная зона, локальная отслойка сетчатки вдоль демаркационной линии"
23,извитые,нормальная,"ДЗН бледный, монотонный, границы четкие, артерии полнокровны, извиты, от ДЗН в наружной половине отслойка сетчатки в виде полиморфных складок сетчатки, множественные преретинальные помутнения"
94,нормальная,извитые,"Д.з.н. – бледно-розовый, контуры четкие, в макулярной области субретинальный желтоватый экссудат, в виде «фигуры звезды», субретинальный экссудат в парамакулярной области и на средней периферии, в наружном квадранте извитый сосуд с множественными аневризмами Д.з.н. – бледно-розовый, контуры четкие, в макулярной области субретинальный желтоватый экссудат, в виде «фигуры звезды», субретинальный экссудат в парамакулярной области и на средней периферии, в наружном квадранте извитый сосуд с множественными аневризмами"
234,извитые,нормальная,"ДЗН – бледный, монотонный, границы четкие, перипапиллярная пигментация, височный сдвиг сосудов, от ДЗН к наружной периферии идет складка сетчатки, концентрически закручивающ. на периферии , атрофические хориоретинальные очаги ДЗН – бледный, монотонный, границы четкие, перипапиллярная пигментация, височный сдвиг сосудов, от ДЗН к наружной периферии идет складка сетчатки, концентрически закручивающ. на периферии , атрофические хориоретинальные очаги"
307,извитые,нормальная,"ДЗН бледный, монотонный, тракционно деформирован, артерии извиты, полнокровны, от ДЗН к наружной периферии идет складка сетчатки, на периферии остаточные аваскулярные зоны"
318,извитые,нормальная,"ДЗН бледный, деколорирован с височной стороны, вены умеренно полнокровны, артерии извиты, центральные рефлексы сглажены, сетчатка бледная, на периферии узкие аваскулярные зоны сетчатки, зоны продолженного интраретинального роста сосудов"
406,извитые,нормальная,"ДЗН белдно-розовый, границы стушеваны, сосуды извиты, субретинальный эксудат по всему глазному дну, распрастраненная отслойка сетчатки на периферии, на периферии в наружном и нижнем сегментах множественные ангиоматозно расширенные сосуды и микроаневризмы"
581,нормальная,извитые,"ДЗН бледно-розовый, с четкими границами, ход и калибр сосудов не изменен, макулярный рефлекс четкий, на средней периферии в меридиане 9 час ангиоматозные узелочки с расширенными, извитыми приводящими сосудами. остальная сетчатка без очаговой патологии"
586,нормальная,извитые,"ДЗН бледный, проминирует, границы не прослеживаются, вены значительно расширены, извиты, артерии сужены, артериовенозные шунты, сетчатка в области заднего полюса отечны, эпиретинальный фиброз, точечные интраретинальные кровоизлияния, периферия без особенностей"
719,нормальная,извитые,"ДЗН –бледно-розовый, с четкими границами, сосудистый пучок в центре. Видны мелкие кистозно расширенные и извитые сосуды. Макулярная область – пигментированные очаги, в парамакулярной области- зоны точечного отложения экссудата. На периферии отслойка сетчатки на 7-13 часах в виде «пузырей». ДЗН –бледно-розовый, с четкими границами, сосудистый пучок в центре. Видны мелкие кистозно расширенные и извитые сосуды. Макулярная область – пигментированные очаги, в парамакулярной области- зоны точечного отложения экссудата. На периферии отслойка сетчатки на 7-13 часах в виде «пузырей»."


В большинстве случаев расхождения действительно имеют место быть, однако, по моим оценкам есть неправильные расхождения в описаниях: 4, 406, 719, 775, 1012

Во всех этих случаях должны быть извитые как вены, так и артерии, так как в описаниях в общем говорится об извитости сосудов.

In [79]:
valid_descr.loc[[4, 406, 719, 775, 1012]][['vessels_art_turtuosity', 'vessels_vein_turtuosity',  'text']]

,vessels_art_turtuosity,vessels_vein_turtuosity,text
4,нормальная,извитые,"ДЗН бледно-розовый, границы четкие, артерии несколько сужены, сосуды несколько извиты, смещены в височную сторону. Макуляные рефлексы сглажены, на периферии в наружних и нижних отделах (зона 2,3) аваскулярная зона, локальная отслойка сетчатки вдоль демаркационной линии"
406,извитые,нормальная,"ДЗН белдно-розовый, границы стушеваны, сосуды извиты, субретинальный эксудат по всему глазному дну, распрастраненная отслойка сетчатки на периферии, на периферии в наружном и нижнем сегментах множественные ангиоматозно расширенные сосуды и микроаневризмы"
719,нормальная,извитые,"ДЗН –бледно-розовый, с четкими границами, сосудистый пучок в центре. Видны мелкие кистозно расширенные и извитые сосуды. Макулярная область – пигментированные очаги, в парамакулярной области- зоны точечного отложения экссудата. На периферии отслойка сетчатки на 7-13 часах в виде «пузырей». ДЗН –бледно-розовый, с четкими границами, сосудистый пучок в центре. Видны мелкие кистозно расширенные и извитые сосуды. Макулярная область – пигментированные очаги, в парамакулярной области- зоны точечного отложения экссудата. На периферии отслойка сетчатки на 7-13 часах в виде «пузырей»."
775,нормальная,извитые,"ДЗН бледный, колобома ДЗН, сосуды на периферии умеренно извиты, артерии сужены, центральные рефлексы практически отсутствуют, на периферии в наружном отделе множественные атрофические очаги с отложением пигмента, сетчатка бледная"
1012,нормальная,извитые,"Д.з.н. – бледно-розовый, контуры четкие, в макулярной области субретинальный желтоватый экссудат, в виде «фигуры звезды», субретинальный экссудат в парамакулярной области и на средней периферии, в наружном квадранте извитые сосуды с множественными аневризмами Д.з.н. – бледно-розовый, контуры четкие, в макулярной области субретинальный желтоватый экссудат, в виде «фигуры звезды», субретинальный экссудат в парамакулярной области и на средней периферии, в наружном квадранте извитые сосуды с множественными аневризмами"


In [80]:
valid_descr.loc[[4, 406, 719, 775, 1012], 'vessels_art_turtuosity'] = 'извитые'
valid_descr.loc[[4, 406, 719, 775, 1012], 'vessels_vein_turtuosity'] = 'извитые'

#### vessels_vein_bifurcation

In [81]:
valid_descr['vessels_vein_bifurcation'] = valid_descr['vessels_vein_bifurcation'].astype(str)
vessels_vein_bifurcation_s = valid_descr['vessels_vein_bifurcation']
np.unique(vessels_vein_bifurcation_s)

array([' под острым углом ', 'nan', 'в норме', 'нормальная',
       'под острым углом'], dtype=object)

Системные значения: 
* нормальная
* под острым углом
* под тупым углом

В данном случае выделено моделью только 3 случае острого угла, хотя с артериями было 4.

In [82]:
valid_descr[valid_descr['vessels_vein_bifurcation']=='под острым углом']['text']

352                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  ДЗН бледно-розовый, монотонный, границы четкие, парапапиллярная пигментация, височный сдвиг сосудов (угол выхода сосудов 20гр), МЗ не дифференцируется, от ДЗН к наружней периферии идет нежный тяж, видимая периферия без особенностей
374                                                                                                                                                                                                                                                                                                        

In [83]:
valid_descr[valid_descr['vessels_vein_bifurcation']==' под острым углом ']['text']

1019    ДЗН бледный, границы нечеткие, сосуды извиты, в МЗ твердый субретинальный эксудат, на периферии отложения субретинального эксудата, единичные ангиоматозно расширенные сосуды и микроаневризмы, пигментированные коагуляты, отграничивающие зоны эксудации ДЗН бледный, границы нечеткие, сосуды извиты, в МЗ твердый субретинальный эксудат, на периферии отложения субретинального эксудата, единичные ангиоматозно расширенные сосуды и микроаневризмы , пигментированные коагуляты, отграничивающие зоны эксудации
Name: text, dtype: object

In [84]:
vessels_vein_bifurcation_s.value_counts()

нормальная            885
под острым углом        3
nan                     1
в норме                 1
 под острым углом       1
Name: vessels_vein_bifurcation, dtype: int64

In [85]:
replacement_dict = {
    "nan": "нормальная",
    "в норме": "нормальная",
    " под острым углом ": "нормальная"
}

vessels_vein_bifurcation_s = vessels_vein_bifurcation_s.replace(replacement_dict)
valid_descr.loc[:, 'vessels_vein_bifurcation'] = vessels_vein_bifurcation_s
valid_descr['vessels_vein_bifurcation'].value_counts()

нормальная          888
под острым углом      3
Name: vessels_vein_bifurcation, dtype: int64

В случаях расхождений значений, на основе текстовых описаний, следует написать бифуркацию == "под острым углом"

In [86]:
valid_descr[valid_descr['vessels_vein_bifurcation'] != valid_descr['vessels_art_bifurcation']][['vessels_art_bifurcation', 'vessels_vein_bifurcation',  'text']]

,vessels_art_bifurcation,vessels_vein_bifurcation,text
352,нормальная,под острым углом,"ДЗН бледно-розовый, монотонный, границы четкие, парапапиллярная пигментация, височный сдвиг сосудов (угол выхода сосудов 20гр), МЗ не дифференцируется, от ДЗН к наружней периферии идет нежный тяж, видимая периферия без особенностей"
373,под острым углом,нормальная,"ДЗН бледный, тракционно деформирован, угол выхода сосудов сужен, макула деформирована, смещена в височную сторону, рефлексы сглажены, на периферии множественные пигментированные сливные коагуляты"
482,под острым углом,нормальная,"ДЗН бледно-розовый, границы четкие, фиброзная складка над ДЗН, угол выхода сосудов уменьшен, МЗ не сформирована, гипопигментация сетчатки, в наружных отделах остаточные аваскулярные зоны"


In [87]:
valid_descr.loc[[352, 373, 482], 'vessels_art_bifurcation'] = 'под острым углом'
valid_descr.loc[[352, 373, 482], 'vessels_vein_bifurcation'] = 'под острым углом'

#### vessels_vein_caliber

In [88]:
valid_descr['vessels_vein_caliber'] = valid_descr['vessels_vein_caliber'].astype(str)
vessels_vein_caliber_s = valid_descr['vessels_vein_caliber']
np.unique(vessels_vein_caliber_s)

array([" нормальный};'}el'{virons inflammatory '",
       'ангиоматозно расширенный', 'ангиоматозноразширенный',
       'значительно расширены', 'нарушен', 'нормальный', 'обеднены',
       'полнокровны', 'полнокровные', 'полнокровный', 'расширенные',
       'расширенный', 'резко расширенный', 'резко суженные',
       'резко сужены', 'суженный', 'утолщенный', 'широкие', 'широкий'],
      dtype=object)

Системные значения: 
* нормальный 
* расширенный 
* суженный

In [89]:
vessels_vein_caliber_s.value_counts()

нормальный                                  722
суженный                                     86
расширенный                                  52
полнокровны                                  11
полнокровные                                  4
полнокровный                                  3
значительно расширены                         1
утолщенный                                    1
нарушен                                       1
широкий                                       1
ангиоматозноразширенный                       1
резко сужены                                  1
расширенные                                   1
ангиоматозно расширенный                      1
резко расширенный                             1
резко суженные                                1
широкие                                       1
обеднены                                      1
 нормальный};'}el'{virons inflammatory '      1
Name: vessels_vein_caliber, dtype: int64

In [90]:
valid_descr[valid_descr['vessels_vein_caliber']==""" нормальный};'}el'{virons inflammatory '"""]['text']

1019    ДЗН бледный, границы нечеткие, сосуды извиты, в МЗ твердый субретинальный эксудат, на периферии отложения субретинального эксудата, единичные ангиоматозно расширенные сосуды и микроаневризмы, пигментированные коагуляты, отграничивающие зоны эксудации ДЗН бледный, границы нечеткие, сосуды извиты, в МЗ твердый субретинальный эксудат, на периферии отложения субретинального эксудата, единичные ангиоматозно расширенные сосуды и микроаневризмы , пигментированные коагуляты, отграничивающие зоны эксудации
Name: text, dtype: object

In [91]:
valid_descr[valid_descr['vessels_vein_caliber']=='широкий']['text']

777    ДЗН бледно-розовый монотонный, границы четкие, ход сосудов не изменен, вены широкие, МЗ и периферия без особенностей
Name: text, dtype: object

In [92]:
valid_descr[valid_descr['vessels_vein_caliber']=='обеднены']['text']

137    ДЗН бледноват, наружные границы стушеваны, от него в наружный отдел идет высокий тяж фибро-васкулярный, крепящийся за хрусталиком в наружном отделе. остальная сетчатка с перераспределением пигмента, арефлексична, обеднена сосудами
Name: text, dtype: object

In [93]:
replacement_dict = {
    "полнокровны": "расширенный",
    "полнокровный": "расширенный",
    "полнокровные": "расширенный",
    "расширенные": "расширенный",
    "значительно расширены": "расширенный",
    "утолщенный": "расширенный",
    "широкий": "расширенный",
    "широкие": "расширенный",
    "резко расширенный": "расширенный",
    "ангиоматозно расширенный": "расширенный",
    "ангиоматозноразширенный": "расширенный",
    "несколько сужены": "суженный",
    "резко суженный": "суженный",
    "резко сужены": "суженный",
    "резко суженные": "суженный",
    "обеднены": "нормальный",
    """ нормальный};'}el'{virons inflammatory '""": "расширенный"
}

vessels_vein_caliber_s = vessels_vein_caliber_s.replace(replacement_dict)
valid_descr.loc[:, 'vessels_vein_caliber'] = vessels_vein_caliber_s
valid_descr['vessels_vein_caliber'].value_counts()

нормальный     723
суженный        88
расширенный     79
нарушен          1
Name: vessels_vein_caliber, dtype: int64

Наблюдается много расхождений калибра вен и сосудов, пока оставим так как есть.

In [94]:
valid_descr[valid_descr['vessels_vein_caliber'] != valid_descr['vessels_art_caliber']][['vessels_art_caliber', 'vessels_vein_caliber',  'text']]

,vessels_art_caliber,vessels_vein_caliber,text
4,суженный,нормальный,"ДЗН бледно-розовый, границы четкие, артерии несколько сужены, сосуды несколько извиты, смещены в височную сторону. Макуляные рефлексы сглажены, на периферии в наружних и нижних отделах (зона 2,3) аваскулярная зона, локальная отслойка сетчатки вдоль демаркационной линии"
23,расширенный,нормальный,"ДЗН бледный, монотонный, границы четкие, артерии полнокровны, извиты, от ДЗН в наружной половине отслойка сетчатки в виде полиморфных складок сетчатки, множественные преретинальные помутнения"
44,нормальный,расширенный,"ДЗН бледно-розовый, границы четкие. сосудистый пучок в центре. вены полнокровны. Эпиретинальные пленки на 5-6ч. Плоский ретиношизис на7 часах. Паравазальная мембрана на 5, 6, 8ч. Паравазальный экссудат на 6, 7 часах периферии. ДЗН бледно-розовый, границы четкие. сосудистый пучок в центре. вены полнокровны. Эпиретинальные пленки на 5-6ч. Плоский ретиношизис на7 часах. Паравазальная мембрана на 5, 6, 8ч. Паравазальный экссудат на 6, 7 часах периферии."
62,суженный,нормальный,"ДЗН бледный, монотонный, границы четкие, сосуды резко сужены, грубые полиморфные изменения всех слоев гл.дна.макулярного и фовеолярного рефлексов нет.Дефекты пигментного и хориокаппиллярного слоя."
67,суженный,нормальный,"ДЗН бледноватый, границы прослеживаются, артерии умеренно сужены, вал вдавления от 2 до 8 час., и старый отрывом от зубчатой линии на 4 – 6 час., старые лазеркоагуляты ДЗН бледноватый, границы прослеживаются, артерии умеренно сужены, вал вдавления от 2 до 8 час., и старый отрывом от зубчатой линии на 4 – 6 час., старые лазеркоагуляты"
...,...,...,...
973,суженный,расширенный,"ДЗН бледно-розовый, овальной формы, границы четкие, назальная часть несколько проминирует в стекловидное тело, глубокая экскавация 0,5 ДД. Сосуды извиты, артерии сужены, вены расширены, МЗ и периферия без особенностей"
1004,нормальный,расширенный,"За флером, ДЗН бледно-розовый, границы четкие, вены полнокровны, макулярный рефлекс сглажен, детали не визуализируются"
1011,суженный,расширенный,"ДЗН бледный, монотонный, границы с внутренней стороны сглажены, артерии несколько сужены, вены полнокровны, макулярные рефлексы сглажены, в МЗ диссоциация пигмента, периферия без особенностей"
1013,суженный,нормальный,"ДЗН блдено-розовый, границы четкие, тракционно деформирован, артерии несколько сужены, ектопия МЗ, на периферии аваскулярные зоны, на 8-10ч локальная отслойка сетчатки, на 8-8:30 2а атрофических хороиретинальных очага, к которым подходят ангиоматозно расширенные сосуды"


### vessels_ratio

In [95]:
valid_descr['vessels_ratio'] = valid_descr['vessels_ratio'].astype(str)
vessels_ratio_s = valid_descr['vessels_ratio']
np.unique(vessels_ratio_s)

array(['-', '.', '.None', '.ov', '/', '/None', '1:1,5', '1:1.5', '2:3',
       ':', ': ', 'nan', 'А:В= 1:1,5', 'А:В=1:1.5',
       'Ангиоматозно расширенные сосуды и микроаневризмы в верхне-наружном, наружном и внутреннем сегментах.',
       'Высокая сосудистая активность', 'Не указано.',
       'Недостаточное кровоснабжение.',
       'Отношение артерий и вен изменено, артерий мало и они сужены.',
       'Сетчатка обеднена сосудами',
       'Сосуды тракционно деформированы в височном направлении.',
       'Сужены резко артерии',
       'Тракционная деформация сосудов в височном направлении.',
       'Умеренно суженные сосуды', 'активные сосуды',
       'артерии и вены бедно выражены', 'артериовенозные шунты',
       'более активные сосуды', 'вены полнокровны', 'височный сдвиг',
       'множественные ангиоматозно расширенные сосуды и микроаневризмы в верхнем отделе',
       'резкое сужение всех сосудов', 'сетка обеднена сосудами',
       'сосуды сужены', 'суженные артерии', 'умеренно по

Сюда что-то моделька выписавала всё подряд, но при этом есть и разумные значения, 1:1.5 и 2:3, при этом мы обсуждали, что нормальным соотношением как раз является 2:3. 

In [96]:
vessels_ratio_s.value_counts()

nan                                                                                                     849
А:В=1:1.5                                                                                                 5
.                                                                                                         3
2:3                                                                                                       2
Умеренно суженные сосуды                                                                                  1
вены полнокровны                                                                                          1
суженные артерии                                                                                          1
Сосуды тракционно деформированы в височном направлении.                                                   1
умеренно полнокровны                                                                                      1
резкое сужение всех сосудов 

Так как в тех случаях, где нормально указано соотношение А:В оно соответствует норме, то присвоим вообще всем значениям показатель нормы. 

In [97]:
valid_descr.loc[:, 'vessels_ratio'] = '2:3'

## Macula

#### macula_macular_reflex

In [98]:
valid_descr['macula_macular_reflex'] = valid_descr['macula_macular_reflex'].astype(str)
macula_macular_reflex_s = valid_descr['macula_macular_reflex']
np.unique(macula_macular_reflex_s)

array([' сглаженный', '/', 'MZ – не дифференцируется', 'nan',
       'subclass alignertractions extended.', 'арефлексична',
       'выраженный', 'выражены', 'гиперпигментирована', 'гипопластична',
       'гладкая', 'гладкий', 'дистрофические желтоватые очаги',
       'желточная', 'кистовидные изменения', 'не нормален',
       'немного расширен', 'нормальный',
       'обширный проминирующий очаг с субретинальным экссудатом',
       'ослаблен', 'ослаблены', 'отсутствует', 'подавлен', 'растушеваны',
       'расширен', 'расширена', 'расширены', 'сглажен', 'сглажена',
       'сглаженные', 'сглаженный', 'сглажены', 'сильно сглажен',
       'складчатый', 'суженый', 'четкие', 'четкий'], dtype=object)

Системные значения: 
* отсутсвует 
* нормальный 
* сглаженный

In [99]:
macula_macular_reflex_s.value_counts()

нормальный                                                 576
гладкий                                                    126
отсутствует                                                 83
сглаженный                                                  27
сглажен                                                     15
сглажена                                                    11
nan                                                         10
сглажены                                                     8
выражены                                                     3
четкий                                                       3
немного расширен                                             2
гиперпигментирована                                          2
растушеваны                                                  1
MZ – не дифференцируется                                     1
выраженный                                                   1
ослаблен                                               

In [100]:
valid_descr[valid_descr['macula_macular_reflex']=='subclass alignertractions extended.']['text']

1019    ДЗН бледный, границы нечеткие, сосуды извиты, в МЗ твердый субретинальный эксудат, на периферии отложения субретинального эксудата, единичные ангиоматозно расширенные сосуды и микроаневризмы, пигментированные коагуляты, отграничивающие зоны эксудации ДЗН бледный, границы нечеткие, сосуды извиты, в МЗ твердый субретинальный эксудат, на периферии отложения субретинального эксудата, единичные ангиоматозно расширенные сосуды и микроаневризмы , пигментированные коагуляты, отграничивающие зоны эксудации
Name: text, dtype: object

In [101]:
replacement_dict = {
    "четкий": "нормальный",
    "четкие": "нормальный",
    "выражены": "нормальный",
    "выраженный": "нормальный",
    "гладкий": "нормальный",
    "гладкая": "нормальный",
    "немного расширен": "нормальный",
    "расширен": "нормальный",
    "расширены": "нормальный",
    "расширена": "нормальный",
    "гиперпигментирована": "нормальный",
    "желточная": "нормальный",
    "nan": "нормальный",
    "/": "нормальный",

    "отсутствует": "отсутствует",
    "MZ – не дифференцируется": "отсутствует",
    "арефлексична": "отсутствует",
    "не нормален": "отсутствует",

    "сглаженный": "сглаженный",
    "сглажен": "сглаженный",
    "сглажена": "сглаженный",
    "сглажены": "сглаженный",
    "сглаженные": "сглаженный",
    "сильно сглажен": "сглаженный",
    "растушеваны": "сглаженный",
    "ослаблен": "сглаженный",
    "ослаблены": "сглаженный",
    "подавлен": "сглаженный",
    "складчатый": "сглаженный",
    " сглаженный": "сглаженный",  # Учитываем ошибочный пробел
    "суженый": "сглаженный",  # Суженный рефлекс можно считать сглаженным

    # Малораспространенные и непонятные описания — относим к "отсутствует"
    "дистрофические желтоватые очаги": "отсутствует",
    "обширный проминирующий очаг с субретинальным экссудатом": "отсутствует",
    "кистовидные изменения": "отсутствует",
    "гипопластична": "отсутствует",
    "subclass alignertractions extended.": "отсутствует",
}


macula_macular_reflex_s = macula_macular_reflex_s.replace(replacement_dict)
valid_descr.loc[:, 'macula_macular_reflex'] = macula_macular_reflex_s
valid_descr['macula_macular_reflex'].value_counts()

нормальный     730
отсутствует     91
сглаженный      70
Name: macula_macular_reflex, dtype: int64

### macula_foveal_reflex

In [102]:
valid_descr['macula_foveal_reflex'] = valid_descr['macula_foveal_reflex'].astype(str)
macula_foveal_reflex_s = valid_descr['macula_foveal_reflex']
np.unique(macula_foveal_reflex_s)

array([' отсутствует', '/', 'nan', "o],' cndigo 'ecoquenc", 'выражен',
       'выраженный', 'выражены', 'гиперпигментирована',
       'гиперпигментированная', 'гладкий', 'деформированы',
       'дистрофические желтоватые очаги', 'крапчатость', 'нормальный',
       'отсутствует', 'растушеваны', 'розовый', 'сглажен', 'сглажена',
       'сглаженные', 'сглаженный', 'сглажены', 'сильно сглажен',
       'суженый', 'темный точечный фокус'], dtype=object)

Системные значения: 
* отсутсвует 
* нормальный 
* сглаженный

In [103]:
macula_foveal_reflex_s.value_counts()

нормальный                         628
гладкий                            101
отсутствует                         84
сглаженный                          27
nan                                 15
сглажен                              9
сглажены                             7
/                                    2
сглажена                             2
сглаженные                           1
деформированы                        1
гиперпигментирована                  1
суженый                              1
выражен                              1
сильно сглажен                       1
розовый                              1
крапчатость                          1
выраженный                           1
 отсутствует                         1
растушеваны                          1
дистрофические желтоватые очаги      1
гиперпигментированная                1
выражены                             1
темный точечный фокус                1
o],' cndigo 'ecoquenc                1
Name: macula_foveal_refle

In [104]:
valid_descr[valid_descr['macula_foveal_reflex']=="o],' cndigo 'ecoquenc"]['text']

1019    ДЗН бледный, границы нечеткие, сосуды извиты, в МЗ твердый субретинальный эксудат, на периферии отложения субретинального эксудата, единичные ангиоматозно расширенные сосуды и микроаневризмы, пигментированные коагуляты, отграничивающие зоны эксудации ДЗН бледный, границы нечеткие, сосуды извиты, в МЗ твердый субретинальный эксудат, на периферии отложения субретинального эксудата, единичные ангиоматозно расширенные сосуды и микроаневризмы , пигментированные коагуляты, отграничивающие зоны эксудации
Name: text, dtype: object

In [105]:
valid_descr[valid_descr['macula_foveal_reflex']=="крапчатость"]['text']

364    ДЗН – бледно розовый с четкими границами. Сосудистый пучок в центре А:В=1:1.5, В макуле кистовидные изменения, крапчатость, ретиношизис? По периферии картина «битого металла» перераспределение пигмента. ДЗН – бледно розовый с четкими границами. Сосудистый пучок в центре А:В=1:1.5, В макуле кистовидные изменения, крапчатость, ретиношизис? По периферии картина «битого металла» перераспределение пигмента.
Name: text, dtype: object

In [106]:
replacement_dict = {
    "нормальный": "нормальный",
    "выражен": "нормальный",
    "выражены": "нормальный",
    "выраженный": "нормальный",
    "гладкий": "нормальный",
    "розовый": "нормальный",
    "гиперпигментирована": "нормальный",
    "гиперпигментированная": "нормальный",

    "отсутствует": "отсутствует",
    " отсутствует": "отсутствует",  # Учитываем пробел в начале
    "nan": "отсутствует",
    "o],' cndigo 'ecoquenc": "отсутствует",  # Ошибочное значение    
    "дистрофические желтоватые очаги": "отсутствует",
    "темный точечный фокус": "отсутствует",
    "крапчатость": "отсутствует",
    
    "сглаженный": "сглаженный",
    "сглажен": "сглаженный",
    "сглажена": "сглаженный",
    "сглажены": "сглаженный",
    "сглаженные": "сглаженный",
    "сильно сглажен": "сглаженный",
    "растушеваны": "сглаженный",
    "деформированы": "сглаженный",
    "суженый": "сглаженный",
}

macula_foveal_reflex_s = macula_foveal_reflex_s.replace(replacement_dict)
valid_descr.loc[:, 'macula_foveal_reflex'] = macula_foveal_reflex_s
valid_descr['macula_foveal_reflex'].value_counts()

нормальный     735
отсутствует    104
сглаженный      50
/                2
Name: macula_foveal_reflex, dtype: int64

Тут модель ошиблась в случае описания 873, должно быть значение рефлексов - "сглаженный", а в 10 описании сложный кейс, и как я понимаю, там уже отсутсвуют какие-либо рефлексы.

In [107]:
valid_descr[valid_descr['macula_foveal_reflex']=="/"]['text']

10                                                                                                                                                                                                      ДЗН бледный, границы размыты, сосуды расширены, извиты, тотольная отслойка сетчатки с субретианальным эксудатом. Ангиоматозные сосуды и микроаневризмы на периферии.Преретинальные кровоизлияния в виде мазков над ангиоматозными сосудами.
873    ДЗН бледно-розовый, границы четкие, сосуды не изменены, парапапиллярно - дистрофический очаг желтоватого цвета, пигментированный, макула - рефлексы сглажены, видимая периферия представляется без очаговой патологии ДЗН бледно-розовый, границы четкие, сосуды не изменены, парапапиллярно - дистрофический очаг желтоватого цвета, пигментированный, макула - рефлексы сглажены, видимая периферия представляется без очаговой патолог ии
Name: text, dtype: object

In [108]:
valid_descr.loc[10, 'macula_foveal_reflex'] = 'отсутствует'

In [109]:
valid_descr.loc[873, 'macula_foveal_reflex'] = 'сглаженный'
valid_descr.loc[873, 'macula_macular_reflex'] = 'сглаженный'

## Подсчёт уникальных комбинаций

Подсчитаем количество уникальных комбинаций значений, без учёта патологий.

In [110]:
valid_descr.columns

Index(['is_valid', 'od_color', 'od_monotone', 'od_size', 'od_shape',
       'od_border', 'od_excavation_size', 'od_excavation_location',
       'od_excavation_ratio', 'od_vessels_location', 'od_pathology',
       'vessels_art_course', 'vessels_art_turtuosity',
       'vessels_art_bifurcation', 'vessels_art_caliber', 'vessels_vein_course',
       'vessels_vein_turtuosity', 'vessels_vein_bifurcation',
       'vessels_vein_caliber', 'vessels_ratio', 'vessels_pathology',
       'macula_macular_reflex', 'macula_foveal_reflex', 'macula_pathology',
       'peripheral_pathology', 'other_diagnosis', 'other_notes', 'text'],
      dtype='object')

In [111]:
import pandas as pd

# Указываем столбцы, по которым считаем уникальные комбинации
columns_to_check = [
    'od_color',
    'od_monotone',
    'od_size',
    'od_shape',
    'od_border',
    'od_excavation_size',
    'od_excavation_location',
    'od_excavation_ratio',
    'od_vessels_location',
    'vessels_art_course',
    'vessels_art_turtuosity',
    'vessels_art_bifurcation',
    'vessels_art_caliber',
    'vessels_vein_course',
    'vessels_vein_turtuosity',
    'vessels_vein_bifurcation',
    'vessels_vein_caliber',
    'vessels_ratio',
    'macula_macular_reflex',
    'macula_foveal_reflex',
]

# Считаем уникальные комбинации и их количество
unique_combinations = valid_descr.groupby(columns_to_check).size().reset_index(name='count')

In [112]:
valid_descr.groupby(columns_to_check).ngroups

28

In [113]:
unique_combinations

,od_color,od_monotone,od_size,od_shape,od_border,od_excavation_size,od_excavation_location,od_excavation_ratio,od_vessels_location,vessels_art_course,vessels_art_turtuosity,vessels_art_bifurcation,vessels_art_caliber,vessels_vein_course,vessels_vein_turtuosity,vessels_vein_bifurcation,vessels_vein_caliber,vessels_ratio,macula_macular_reflex,macula_foveal_reflex,count
0,бледно-розовый,True,больше нормы,правильная,четкие,больше нормы,в центре,0.9,в центре,нормальный,нормальная,нормальная,нормальный,нормальный,нормальная,нормальная,нормальный,2:3,нормальный,нормальный,1
1,бледно-розовый,True,больше нормы,правильная,четкие,нормальный,нижне-внутренний,0.3,в центре,нормальный,нормальная,нормальная,нормальный,нормальный,нормальная,нормальная,нормальный,2:3,нормальный,нормальный,1
2,бледно-розовый,True,нормальный,неправильная,четкие,нормальный,наружный,0.3,в центре,аномальный,нормальная,нормальная,нормальный,нормальный,нормальная,нормальная,нормальный,2:3,нормальный,нормальный,1
3,бледно-розовый,True,нормальный,овальная,четкие,больше нормы,в центре,0.5,в центре,нормальный,извитые,нормальная,суженный,нормальный,извитые,нормальная,расширенный,2:3,нормальный,нормальный,1
4,бледно-розовый,True,нормальный,овальная,четкие,нормальный,в центре,0.8,в центре,нормальный,извитые,нормальная,суженный,нормальный,нормальная,нормальная,нормальный,2:3,сглаженный,нормальный,1
5,бледно-розовый,True,нормальный,овальная,четкие,нормальный,в центре,0.9,в центре,нормальный,извитые,нормальная,суженный,нормальный,нормальная,нормальная,нормальный,2:3,сглаженный,нормальный,1
6,бледно-розовый,True,нормальный,правильная,четкие,больше нормы,в центре,0.3,в центре,нормальный,нормальная,нормальная,нормальный,нормальный,нормальная,нормальная,нормальный,2:3,нормальный,нормальный,1
7,бледно-розовый,True,нормальный,правильная,четкие,больше нормы,в центре,0.5,в центре,нормальный,нормальная,нормальная,нормальный,нормальный,нормальная,нормальная,нормальный,2:3,сглаженный,сглаженный,1
8,бледно-розовый,True,нормальный,правильная,четкие,больше нормы,в центре,0.8,внутренний,нормальный,нормальная,нормальная,нормальный,нормальный,нормальная,нормальная,расширенный,2:3,нормальный,нормальный,1
9,бледно-розовый,True,нормальный,правильная,четкие,больше нормы,в центре,0.9,в центре,нормальный,нормальная,нормальная,нормальный,нормальный,нормальная,нормальная,нормальный,2:3,нормальный,нормальный,2


In [114]:
unique_combinations.shape

(28, 21)

In [115]:
valid_descr.columns

Index(['is_valid', 'od_color', 'od_monotone', 'od_size', 'od_shape',
       'od_border', 'od_excavation_size', 'od_excavation_location',
       'od_excavation_ratio', 'od_vessels_location', 'od_pathology',
       'vessels_art_course', 'vessels_art_turtuosity',
       'vessels_art_bifurcation', 'vessels_art_caliber', 'vessels_vein_course',
       'vessels_vein_turtuosity', 'vessels_vein_bifurcation',
       'vessels_vein_caliber', 'vessels_ratio', 'vessels_pathology',
       'macula_macular_reflex', 'macula_foveal_reflex', 'macula_pathology',
       'peripheral_pathology', 'other_diagnosis', 'other_notes', 'text'],
      dtype='object')

In [125]:
all_columns = ['is_valid', 'od_color', 'od_monotone', 'od_size', 'od_shape',
       'od_border', 'od_excavation_size', 'od_excavation_location',
       'od_excavation_ratio', 'od_vessels_location', 'od_pathology',
       'vessels_art_course', 'vessels_art_turtuosity',
       'vessels_art_bifurcation', 'vessels_art_caliber', 'vessels_vein_course',
       'vessels_vein_turtuosity', 'vessels_vein_bifurcation',
       'vessels_vein_caliber', 'vessels_ratio', 'vessels_pathology',
       'macula_macular_reflex', 'macula_foveal_reflex', 'macula_pathology',
       'peripheral_pathology', 'other_diagnosis', 'other_notes',]


# Считаем уникальные комбинации и их количество
num_unique_combinations = valid_descr[all_columns].drop_duplicates().shape[0]
print(num_unique_combinations)

760
